In [1]:
import requests
import json
import sqlite3
from sqlite3 import Error
import pandas as pd
from datetime import datetime

In [ ]:
# headers = {
#     'x-rapidapi-key': "38bab05df5msh928211553580d9cp181186jsne583ad5bd5cd",
#     'x-rapidapi-host': "free-nba.p.rapidapi.com"
#     }
# page = 1
# results = []
# while True:


#     url = "https://free-nba.p.rapidapi.com/stats"

#     querystring = {"page": page,"per_page":"100"}
#     response = requests.request("GET", url, headers=headers, params=querystring)
#     data = response.json()
#     if len(data['data']) <= 0:
#         break
#     results += data['data']
#     print(response.json())
#     break

#     page += 1

#     print(response.json())



In [ ]:
# query nba api for all team data
import requests

url = "https://free-nba.p.rapidapi.com/teams"

querystring = {"page":"0"}

headers = {
    'x-rapidapi-key': "38bab05df5msh928211553580d9cp181186jsne583ad5bd5cd",
    'x-rapidapi-host': "free-nba.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)
# store response in variable
data = response.json()

print(data)

In [ ]:
# write(dump) the json data to a json file
with open('team_data.json', 'w') as json_file:
    json.dump(data, json_file, indent=2)
    json_file.close()

In [ ]:
# load data using Python module into var data
with open('team_data.json', 'r') as f:
    data = json.loads(f.read())
    

In [ ]:
# normalize data as it's nested under 'data' and create dataframe
team_data = pd.json_normalize(data, record_path=['data'])
# send df to csv
team_data.to_csv(r'team_data.csv', index=False)

In [ ]:

# # found how to create db in sqlite3 here:  https://www.sqlitetutorial.net/sqlite-python/insert/

# def create_connection(db_file):
#     """ create a database connection to a SQLite database """
#     conn = None
#     try:
#         conn = sqlite3.connect(db_file)
#         c = conn.cursor()
#         print(sqlite3.version)
#         # this is extra code to create table and load csv into table.  Was testing here
#         c.execute('''CREATE TABLE players (PLAYER_NAME text, TEAM_ID text, PLAYER_ID text, SEASON int)''')
#         players = pd.read_csv('players.csv')
#         players.to_sql('players', conn, if_exists='append', index=False)
#         c.execute('''SELECT * FROM players''').fetchall()

#     except Error as e:
#         print(e)
#     finally:
#         if conn:
#             conn.close()


# if __name__ == '__main__':
#     create_connection(r"nba_test.db")

In [2]:
# another method of creating db and tables but without function: https://mungingdata.com/sqlite/create-database-load-csv-python/
import sqlite3
# create connection to build new db if nonexistent and cursor to execute queries
conn = sqlite3.connect('nba_test')
c = conn.cursor()

In [ ]:
# create table players **testing to see if works**
c.execute('''CREATE TABLE players (PLAYER_NAME text, TEAM_ID text, PLAYER_ID text, SEASON int)''')

In [ ]:
# using pandas to read csv then push data to table created in last cell
players = pd.read_csv('players.csv')
players.to_sql('players', conn, if_exists='append', index=False)


In [ ]:
# querying table to verify creation
c.execute('''SELECT * FROM players LIMIT 5''').fetchall()

In [ ]:
# create rest of tables
players = pd.read_csv('players.csv')
players.to_sql('players', conn, if_exists='append', index=False)
teams = pd.read_csv('teams.csv')
teams.to_sql('teams', conn, if_exists='append', index=False)
games = pd.read_csv('games.csv')
games.to_sql('games', conn, if_exists='append', index=False)
games_details = pd.read_csv('games_details.csv')
games_details.to_sql('games_details', conn, if_exists='append', index=False)
kobe = pd.read_csv('kobe_data.csv')
kobe.to_sql('kobe', conn, if_exists='append', index=False)
lebron = pd.read_csv('lebron_data.csv')
lebron.to_sql('lebron', conn, if_exists='append', index=False)
team_data = pd.read_csv('team_data.csv')
team_data.to_sql('team_data', conn , if_exists='append', index=False)

In [ ]:
# create sqlite table from Kobe csv
kobe = pd.read_csv('kobe_data.csv')
kobe.to_sql('kobe', conn, if_exists='append', index=False)

In [ ]:
team_data = pd.read_csv('team_data.csv')
team_data.to_sql('team_data', conn , if_exists='append', index=False)

In [ ]:
# create sqlite table from Lebron csv
lebron = pd.read_csv('lebron_data.csv')
lebron.to_sql('lebron', conn, if_exists='append', index=False)

In [ ]:
#  checking lebron tabe that was just imported
c.execute('''SELECT * FROM lebron LIMIT 5''').fetchall()


In [ ]:
c.execute('''SELECT * FROM kobe LIMIT 5''').fetchall()

In [ ]:
# Kobe ID is 977, Lebron ID is 2544
# testing select statement for filtering by player id
c.execute('''SELECT * FROM players WHERE PLAYER_ID = 2544''').fetchall()

In [ ]:
# testing the join statement to get actual game date from games table
c.execute('''SELECT games.game_id, game_date_est, player_name, fg_pct, fg3_pct, REB, AST, STL, BLK,PF,PTS,PLUS_MINUS FROM games LEFT JOIN games_details ON games.game_id = games_details.game_id WHERE PLAYER_ID = 2544 LIMIT 5'').fetchall()

In [ ]:
# testing select statement to query additional stats and filtering by player id
# INNER JOIN games ON games.game_id = games_details.game_id  DIDN'T WORK
c.execute('''SELECT [game.date] from kobe LIMIT 10''').fetchall()


In [ ]:
c.execute('''UPDATE kobe SET [game.date] = DATE_FORMAT(STR_TO_DATE([game.date], '%Y-%m-%dT%H:%i:%s.000Z'),'%Y-%m-%d')''')

In [ ]:
# testing select statement to query additional stats and filtering by player id in a DataFrame
pd.read_sql('''SELECT TEAM_ABBREVIATION, PLAYER_NAME, FG_PCT, FG3_PCT, FT_PCT, PTS FROM games_details WHERE PLAYER_ID = 2544''', conn)


In [ ]:
lebron_data = pd.read_sql('''SELECT teams.abbreviation AS opposing_team, visitor_team_id, team_abbreviation, games.game_id, game_date_est, player_name, fg_pct, fg3_pct, REB, AST, STL, BLK,PF,PTS,PLUS_MINUS FROM games LEFT JOIN games_details ON games.game_id = games_details.game_id LEFT JOIN teams ON teams.team_id = games.VISITOR_TEAM_ID WHERE PLAYER_ID = 2544''', conn)
lebron_data

In [ ]:
# this is from the original csv and it's not complete for Kobe
kobe_data = pd.read_sql('''SELECT home_team_id, visitor_team_id, team_abbreviation, games_details.team_id, games.game_id, game_date_est, player_name, fg_pct, fg3_pct, REB, AST, STL, BLK,PF,PTS,PLUS_MINUS FROM games LEFT JOIN games_details ON games.game_id = games_details.game_id LEFT JOIN teams ON teams.team_id = games.VISITOR_TEAM_ID WHERE PLAYER_ID = 977''', conn)
kobe_data

In [ ]:
teamdata = pd.read_sql('''SELECT * FROM team_data''', conn)
teamdata

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# query using new Kobe_data.csv from Kim
#  Available Headers: id,ast,blk,dreb,min,pts,reb,stl,turnover,game.id,game.date,game.home_team_id,game.home_team_score,game.period,game.postseason,game.season,game.status,game.time,game.visitor_team_id,game.visitor_team_score,player.id,player.first_name,player.last_name,player.position,player.team_id,team.id,team.abbreviation,team.city,team.conference,team.division,team.full_name,team.name
# found solution for datetime formatting here: https://www.tutlane.com/tutorial/sqlite/sqlite-strftime-function
Kobe_stats = pd.read_sql('''SELECT [game.id] as "game_id", ast,blk,min,pts,reb,stl,turnover, [game.date] as "date", strftime('%Y', [game.date]) as "Year",[game.home_team_id] as "home_team_id",[game.home_team_score] as "home_team_score", [game.visitor_team_id] as "visitor_team_id",[game.visitor_team_score] as "visitor_team_score" FROM kobe ORDER BY date ASC''', conn)
# Kobe_stats['game.date'] = pd.to_datetime(Kobe_stats['game.date']).dt.date 
Kobe_stats

In [ ]:
c.execute('''SELECT [game.id] as "game_id", ast,blk,min,pts,reb,stl,turnover, [game.date] as "date", strftime('%Y', [game.date]) as "Year",[game.home_team_id] as "home_team_id",[game.home_team_score] as "home_team_score", [game.visitor_team_id] as "visitor_team_id",[game.visitor_team_score] as "visitor_team_score" FROM kobe ORDER BY date ASC''').fetchall()

In [ ]:
# c.execute('''SELECT [game.id] as "game_id" FROM kobe''').fetchall()
# I suspect there are duplicates in the table
kobe_total_game_id = c.execute('''SELECT COUNT([game.id]) FROM kobe''').fetchall()
kobe_unique_game_id = c.execute('''SELECT COUNT(DISTINCT [game.id]) FROM kobe''').fetchall()
print(f'There are {kobe_total_game_id} total records in the Kobe table')
print(f'There are {kobe_unique_game_id} unique records in the Kobe table')

In [7]:
# sum of points for Kobe per year
# got solution on how to number each row here:  https://www.sqltutorial.org/sql-window-functions/sql-row_number/

kobe_pts = c.execute('''SELECT ROW_NUMBER() OVER (
    ORDER BY strftime('%Y', [game.date])
    )row_num,
    strftime('%Y', [game.date]) as Year, SUM(pts) 
    FROM kobe 
    WHERE pts IS NOT NULL 
    GROUP BY Year''').fetchall()
kobe_pts

[(1, '1996', 106.0),
 (2, '1997', 984.0),
 (3, '1998', 827.0),
 (4, '1999', 1489.0),
 (5, '2000', 2563.0),
 (6, '2001', 2160.0),
 (7, '2002', 2680.0),
 (8, '2003', 2558.0),
 (9, '2004', 2297.0),
 (10, '2005', 1999.0),
 (11, '2006', 2883.0),
 (12, '2007', 2605.0),
 (13, '2008', 2932.0),
 (14, '2009', 3047.0),
 (15, '2010', 2536.0),
 (16, '2011', 1598.0),
 (17, '2012', 2751.0),
 (18, '2013', 1313.0),
 (19, '2014', 698.0),
 (20, '2015', 584.0),
 (21, '2016', 661.0)]

In [8]:
# sum of points for Lebron per year
lebron_pts = c.execute('''SELECT ROW_NUMBER() OVER (
    ORDER BY strftime('%Y', [game.date])
    )row_num, strftime('%Y', [game.date]) as Year, SUM(pts) 
    FROM lebron 
    WHERE pts IS NOT NULL 
    GROUP BY Year''').fetchall()
lebron_pts

[(1, '2003', 647.0),
 (2, '2004', 1715.0),
 (3, '2005', 2317.0),
 (4, '2006', 2821.0),
 (5, '2007', 2589.0),
 (6, '2008', 2730.0),
 (7, '2009', 2917.0),
 (8, '2010', 2426.0),
 (9, '2011', 1910.0),
 (10, '2012', 3010.0),
 (11, '2013', 2634.0),
 (12, '2014', 2604.0),
 (13, '2015', 2357.0),
 (14, '2016', 2471.0),
 (15, '2017', 2803.0),
 (16, '2018', 2926.0),
 (17, '2019', 1381.0),
 (18, '2020', 1576.0),
 (19, '2021', 925.0)]

In [ ]:
 conn.close()

In [ ]:
pd.read_sql('''SELECT * FROM games_details LIMIT 20''', conn)